In [1]:
from qiskit import IBMQ
import qiskit
qiskit.__qiskit_version__

{'qiskit-terra': '0.10.0',
 'qiskit-aer': '0.3.2',
 'qiskit-ignis': '0.2.0',
 'qiskit-ibmq-provider': '0.3.3',
 'qiskit-aqua': '0.6.1',
 'qiskit': '0.13.0'}

In [2]:
provider = IBMQ.load_account()
#change the provider based on your Qiskit account
#provider = IBMQ.get_provider(hub='ibm-q-ncsu', group='nc-state', project='on-boarding')
#if you do not have access to the twenty qubit machine, use this publically available 15qubit 'ibmq_16_melbourne' machine instead
provider = IBMQ.get_provider(hub='ibm-q', group='open', project='main')
provider.backends()

[<IBMQSimulator('ibmq_qasm_simulator') from IBMQ(hub='ibm-q', group='open', project='main')>,
 <IBMQBackend('ibmqx2') from IBMQ(hub='ibm-q', group='open', project='main')>,
 <IBMQBackend('ibmq_16_melbourne') from IBMQ(hub='ibm-q', group='open', project='main')>,
 <IBMQBackend('ibmq_vigo') from IBMQ(hub='ibm-q', group='open', project='main')>,
 <IBMQBackend('ibmq_ourense') from IBMQ(hub='ibm-q', group='open', project='main')>,
 <IBMQBackend('ibmq_london') from IBMQ(hub='ibm-q', group='open', project='main')>,
 <IBMQBackend('ibmq_burlington') from IBMQ(hub='ibm-q', group='open', project='main')>,
 <IBMQBackend('ibmq_essex') from IBMQ(hub='ibm-q', group='open', project='main')>,
 <IBMQBackend('ibmq_armonk') from IBMQ(hub='ibm-q', group='open', project='main')>]

In [3]:
from qiskit import *
from qiskit.compiler import transpile
from qiskit.visualization import plot_histogram
from qiskit.tools.monitor import job_monitor, backend_monitor, backend_overview
from qiskit.providers.aer import noise
import math

#Include our assertion function
from qiskit.compiler.assertion import classical_assertion, superposition_assertion, entanglement_assertion, calcSuccessrate

In [4]:
#we used ibmq_20_tokyo for our experiment, however, that backend has retired. So we can use the 'ibmq_16_melbourne' instead
device = provider.get_backend('ibmq_16_melbourne')
backend_monitor(device)
device

ibmq_16_melbourne
Configuration
-------------
    n_qubits: 15
    operational: True
    status_msg: active
    pending_jobs: 6
    backend_version: 2.0.0
    basis_gates: ['id', 'u1', 'u2', 'u3', 'cx']
    local: False
    simulator: False
    url: None
    memory: True
    max_experiments: 75
    online_date: 2018-11-06T05:00:00+00:00
    quantum_volume: None
    allow_object_storage: True
    max_shots: 8192
    credits_required: True
    open_pulse: False
    allow_q_circuit: False
    n_registers: 1
    backend_name: ibmq_16_melbourne
    description: 15 qubit device
    conditional: False
    coupling_map: [[0, 1], [0, 14], [1, 0], [1, 2], [1, 13], [2, 1], [2, 3], [2, 12], [3, 2], [3, 4], [3, 11], [4, 3], [4, 5], [4, 10], [5, 4], [5, 6], [5, 9], [6, 5], [6, 8], [7, 8], [8, 6], [8, 7], [8, 9], [9, 5], [9, 8], [9, 10], [10, 4], [10, 9], [10, 11], [11, 3], [11, 10], [11, 12], [12, 2], [12, 11], [12, 13], [13, 1], [13, 12], [13, 14], [14, 0], [14, 13]]
    sample_name: albatross
    

<IBMQBackend('ibmq_16_melbourne') from IBMQ(hub='ibm-q', group='open', project='main')>

In [5]:
transQcList = []

pi = math.pi

In [6]:
def entangleSWAP():
    q = QuantumRegister(5)
    c = ClassicalRegister(4)
    circuit = QuantumCircuit(q,c)
    circuit.h(q[1])
    circuit.cx(q[1],q[0])
    circuit.h(q[3])
    circuit.cx(q[3],q[2])

    circuit.cx(q[2],q[1])
    circuit.h(q[2])
    circuit.h(q[1])
    circuit.cx(q[2],q[1])
    circuit.h(q[2])
    circuit.h(q[1])

    circuit.cx(q[3],q[2])
    circuit.h(q[3])
    circuit.h(q[2])
    circuit.cx(q[3],q[2])
    circuit.h(q[3])
    circuit.h(q[2])

    circuit.cx(q[3],q[2])
    circuit.h(q[2])

    circuit.h(q[0])
    circuit.cx(q[2],q[0])
    circuit.h(q[0])

    circuit.h(q[2])
    circuit.cx(q[3],q[2])
    circuit.h(q[3])
    circuit.h(q[2])
    circuit.cx(q[3],q[2])
    circuit.h(q[3])
    circuit.h(q[2])
    circuit.cx(q[3],q[2])
    for i in range(4):
        circuit.measure(q[i],c[i])

    trans_qc = transpile(circuit, backend=device, optimization_level = 3)
    num_cnots = trans_qc.count_ops()['cx']
    return trans_qc, num_cnots


In [7]:
def entangleSWAP_init_assertion():
    q = QuantumRegister(5)
    c = ClassicalRegister(4)
    circuit = QuantumCircuit(q,c)
    circuit.h(q[1])
    circuit.cx(q[1],q[0])
    circuit.h(q[3])
    circuit.cx(q[3],q[2])

    #assertion:
    qubitList = [q[2], q[3]]
    success = entanglement_assertion(circuit, qubitList, flag = 0)

    circuit.cx(q[2],q[1])
    circuit.h(q[2])
    circuit.h(q[1])
    circuit.cx(q[2],q[1])
    circuit.h(q[2])
    circuit.h(q[1])

    circuit.cx(q[3],q[2])
    circuit.h(q[3])
    circuit.h(q[2])
    circuit.cx(q[3],q[2])
    circuit.h(q[3])
    circuit.h(q[2])

    circuit.cx(q[3],q[2])
    circuit.h(q[2])

    circuit.h(q[0])
    circuit.cx(q[2],q[0])
    circuit.h(q[0])

    circuit.h(q[2])
    circuit.cx(q[3],q[2])
    circuit.h(q[3])
    circuit.h(q[2])
    circuit.cx(q[3],q[2])
    circuit.h(q[3])
    circuit.h(q[2])
    circuit.cx(q[3],q[2])

    for i in range(4):
        circuit.measure(q[i],c[i])

    trans_qc = transpile(circuit, backend=device, optimization_level = 3)
    num_cnots = trans_qc.count_ops()['cx']
    return trans_qc, num_cnots


In [8]:
def entangleSWAP_output_assertion():
    q = QuantumRegister(5)
    c = ClassicalRegister(4)
    circuit = QuantumCircuit(q,c)
    circuit.h(q[1])
    circuit.cx(q[1],q[0])
    circuit.h(q[3])
    circuit.cx(q[3],q[2])


    circuit.cx(q[2],q[1])
    circuit.h(q[2])
    circuit.h(q[1])
    circuit.cx(q[2],q[1])
    circuit.h(q[2])
    circuit.h(q[1])

    circuit.cx(q[3],q[2])
    circuit.h(q[3])
    circuit.h(q[2])
    circuit.cx(q[3],q[2])
    circuit.h(q[3])
    circuit.h(q[2])

    circuit.cx(q[3],q[2])
    circuit.h(q[2])

    circuit.h(q[0])
    circuit.cx(q[2],q[0])
    circuit.h(q[0])

    circuit.h(q[2])
    circuit.cx(q[3],q[2])
    circuit.h(q[3])
    circuit.h(q[2])
    circuit.cx(q[3],q[2])
    circuit.h(q[3])
    circuit.h(q[2])
    circuit.cx(q[3],q[2])

    #assertion:
    qubitList = [q[0], q[2]]
    success = entanglement_assertion(circuit, qubitList, flag = 0)

    for i in range(4):
        circuit.measure(q[i],c[i])

    trans_qc = transpile(circuit, backend=device, optimization_level = 3)
    num_cnots = trans_qc.count_ops()['cx']
    return trans_qc, num_cnots

In [9]:
# def entnagleSWAP_output_assertion2():
#     q = QuantumRegister(5)
#     c = ClassicalRegister(4)
#     circuit = QuantumCircuit(q,c)
#     circuit.h(q[1])
#     circuit.cx(q[1],q[0])
#     circuit.h(q[3])
#     circuit.cx(q[3],q[2])


#     circuit.cx(q[2],q[1])
#     circuit.h(q[2])
#     circuit.h(q[1])
#     circuit.cx(q[2],q[1])
#     circuit.h(q[2])
#     circuit.h(q[1])
# #
#     circuit.cx(q[3],q[2])
#     circuit.h(q[3])
#     circuit.h(q[2])
#     circuit.cx(q[3],q[2])
#     circuit.h(q[3])
#     circuit.h(q[2])

#     circuit.cx(q[3],q[2])
#     circuit.h(q[2])

#     circuit.h(q[0])
#     circuit.cx(q[2],q[0])
#     circuit.h(q[0])

#     circuit.h(q[2])
#     circuit.cx(q[3],q[2])
#     circuit.h(q[3])
#     circuit.h(q[2])
#     circuit.cx(q[3],q[2])
#     circuit.h(q[3])
#     circuit.h(q[2])
#     circuit.cx(q[3],q[2])

#     #assertion:
#     qubitList = [q[1], q[3]]
#     success = entanglement_assertion(circuit, qubitList, flag = 0)

#     for i in range(4):
#         circuit.measure(q[i],c[i])

#     trans_qc = transpile(circuit, backend=device, optimization_level = 3)

#     num_cnots = trans_qc.count_ops()['cx']
#     return trans_qc, num_cnots

In [10]:
def entangleSWAPExperiment():
    #This function run the transpiler multiple times and selects the mapping with fewest SWAP gates
    min_cnots = -1
    num_cnots_original = -1
    for i in range(10):
        trans_qc, num_cnots = entangleSWAP()
        if min_cnots < 0:
            min_cnots = num_cnots
        elif (num_cnots < min_cnots) and (min_cnots >= 0):
            min_cnots = num_cnots
    while 1:
        trans_qc, num_cnots = entangleSWAP()
        if num_cnots == min_cnots:
            print("\nnumber of CNOTs in circuit without assertion: ", num_cnots)
            num_cnots_original = num_cnots
            transQcList.append(trans_qc)
            break

    print("\nassertion at the initialization stage:")
    min_cnots = -1
    for i in range(20):
        trans_qc, num_cnots = entangleSWAP_init_assertion()
        if min_cnots < 0:
            min_cnots = num_cnots
        elif (num_cnots < min_cnots) and (min_cnots >= 0):
            min_cnots = num_cnots
    while 1:
        trans_qc, num_cnots = entangleSWAP_init_assertion()
        if num_cnots == min_cnots:
            print("number of CNOTs in circuit with assertion: ", num_cnots)
            percent = abs(1 - num_cnots_original/num_cnots) * 100
            print("    Percentage of the assertion circuit size in total size: ", percent, "%")
            if percent > 20:
                print("    Warning: assertion circuit size is comparable to circuit size, may lead to lower success rate.")
            transQcList.append(trans_qc)
            break
            
    print("\nassertion at the initialization stage:")
    min_cnots = -1
    for i in range(20):
        trans_qc, num_cnots = entangleSWAP_output_assertion()
        if min_cnots < 0:
            min_cnots = num_cnots
        elif (num_cnots < min_cnots) and (min_cnots >= 0):
            min_cnots = num_cnots
    while 1:
        trans_qc, num_cnots = entangleSWAP_output_assertion()
        if num_cnots == min_cnots:
            print("number of CNOTs in circuit with assertion: ", num_cnots)
            percent = abs(1 - num_cnots_original/num_cnots) * 100
            print("    Percentage of the assertion circuit size in total size: ", percent, "%")
            if percent > 20:
                print("    Warning: assertion circuit size is comparable to circuit size, may lead to lower success rate.")
            transQcList.append(trans_qc)
            break

In [11]:
entangleSWAPExperiment()


number of CNOTs in circuit without assertion:  14

assertion at the initialization stage:
number of CNOTs in circuit with assertion:  20
Percentage of the assertion circuit size in total size:  30.000000000000004 %

assertion at the initialization stage:
number of CNOTs in circuit with assertion:  16
    Percentage of the assertion circuit size in total size:  12.5 %


In [12]:
shots = 8192
#execute the job on real device
job = execute(transQcList, backend=device, shots=shots)
job_monitor(job)
res = job.result()

Job Status: job has successfully run


Success rate of SWAP protocol without entanglement assertion:

In [13]:
calcSuccessrate(res.get_counts(0), ['0000', '0101', '1010', '1111'], 0)

total_count =  8192 success_count =  2351 success_rate =  28.69873046875 %


Success rate of SWAP protocol with entanglement assertion at the initialization state:

In [14]:
calcSuccessrate(res.get_counts(1), ['0 0000', '0 0101', '0 1010', '0 1111'], 1)

total_count =  6531 success_count =  1363 success_rate =  20.86969836165978 %


Success rate of SWAP protocol with entanglement assertion at the output state:

In [15]:
calcSuccessrate(res.get_counts(2), ['0 0000', '0 0101', '0 1010', '0 1111'], 1)

total_count =  5127 success_count =  2073 success_rate =  40.433001755412526 %
